In [49]:
# from keras.optimizers import Adam
from keras.layers import InputLayer,Dense,Flatten,BatchNormalization
from keras.applications import vgg16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

import numpy as np
import os


In [50]:
from keras.metrics import Precision,Recall

In [51]:
from sklearn.metrics import f1_score

In [4]:
datagen = ImageDataGenerator()

In [5]:
base_dir = "F:/DATASET/hindi Devnagari handwritten Dataset"

In [6]:
train_base_dir = os.path.join(base_dir,"Train")
test_base_dir = os.path.join(base_dir,"Test")

In [17]:
train_datagen = datagen.flow_from_directory(directory=train_base_dir,
                                            target_size=(32,32),
                                           color_mode="rgb",
                                           batch_size=100)

Found 78200 images belonging to 46 classes.


In [20]:
test_datagen = datagen.flow_from_directory(directory=test_base_dir,
                                          target_size=(32,32),
                                          color_mode="rgb",batch_size=100)

Found 13800 images belonging to 46 classes.


In [52]:
conv_base = vgg16.VGG16(include_top=False,input_shape=(32,32,3))

In [37]:
# conv_base.trainable = False

In [55]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [54]:
conv_base.layers[4].trainable = True
conv_base.layers[8].trainable = True

In [56]:
def unfreeze_layers(layer_name):

  trainable = False

  for layer in conv_base.layers:

    if layer.name == layer_name:
      trainable = True

    if trainable:
      layer.trainable = True
    else:
      layer.trainable = False

  return conv_base

In [41]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [57]:
def build_network():
    network = Sequential()
    conv_base = unfreeze_layers("block5_conv1")
    network.add(conv_base)
    network.add(Flatten())
    network.add(Dense(units = 512,activation= "relu" ,kernel_initializer = "glorot_uniform"))
    network.add(Dense(units = 46,activation = "softmax",kernel_initializer = "glorot_uniform",))
    network.compile(optimizer="adam" , loss= "categorical_crossentropy" ,metrics = ["accuracy" ,Precision(),Recall()])
    return network

In [58]:
network = build_network()

In [59]:
network.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_4 (Flatten)         (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 512)               262656    
                                                                 
 dense_9 (Dense)             (None, 46)                23598     
                                                                 
Total params: 15,000,942
Trainable params: 7,365,678
Non-trainable params: 7,635,264
_________________________________________________________________


In [60]:
network.fit(train_datagen,validation_data=test_datagen,epochs=2)

Epoch 1/2
  8/782 [..............................] - ETA: 15:09 - loss: 7.8395 - accuracy: 0.0188 - precision_4: 0.0183 - recall_4: 0.0037

KeyboardInterrupt: 